In [5]:
pip install langchain openai sentence-transformers pymupdf faiss-gpu dill

  Using cached langchain-0.1.17-py3-none-any.whl (867 kB)
  Using cached openai-1.25.1-py3-none-any.whl (312 kB)
  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached PyMuPDF-1.24.2-cp310-none-manylinux2014_x86_64.whl (3.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
  Using cached dataclasses_json-0.6.5-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langchain_community-0.0.36-py3-none-any.whl (2.0 MB)
  Using cached langchain_core-0.1.50-py3-none-any.whl (302 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
  Using cached langsmith-0.1.53-py3-none-any.whl (116 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap
import torch

In [ ]:
torch.device('cpu')

In [7]:
class Runnable:
    def run(self, *args, **kwargs):
        raise NotImplementedError("Subclass must implement abstract method")

In [8]:

def load_pdf_data(file_path):
    loader = PyMuPDFLoader(file_path=file_path)
    docs = loader.load()
    return docs

In [9]:
# Responsible for splitting the documents into several chunks

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks

In [10]:
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'},
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding
        }
    )

def create_embeddings(chunks, embedding_model, storing_path = "C:\\Users\\nehan\\vectorstore"):


    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    return vectorstore

In [11]:
template = """
### System:
You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
just say you don't know. Don't try to make up an answer.

### Context:
{context}

### User:
{question}

### Response:
"""

In [12]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )

In [13]:
def get_response(query, chain):
    response = chain({'query': query})

    wrapped_text = textwrap.fill(response['result'], width=100)
    print(wrapped_text)

In [14]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

In [15]:
class OpenAIWrapper(Runnable):
    def __init__(self, openai_instance):
        self.openai_instance = openai_instance

    def run(self, *args, **kwargs):
        return self.openai_instance.run(*args, **kwargs)

In [16]:
import os
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = "Paste Your API Key Here"


In [17]:
# Loading openai
llm = OpenAI()

# Loading the Embedding Model
embed = load_embedding_model(model_path = "all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# loading and splitting the documents
docs = load_pdf_data(file_path=r"/content/COI.pdf")
documents = split_docs(documents=docs)

# creating vectorstore
vectorstore = create_embeddings(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

In [21]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template(template)

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [24]:
get_response("tell me about right of self-defence", chain)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The Constitution of India protects the rights of individuals to safeguard public property and to
abjure violence. This includes the right of self-defence, where a person has the right to protect
themselves and their property from harm or danger. However, this right is limited to the extent of
the force necessary to protect oneself and cannot be used as an excuse for committing violence.
